In [1]:
from google.colab import drive
import os
drive.mount('/content/gdrive/', force_remount=True)
os.chdir("/content/gdrive/MyDrive/Usc/EE641/Project/")
print("current working directory is ", os.getcwd())
!pip install yacs

Mounted at /content/gdrive/
current working directory is  /content/gdrive/MyDrive/Usc/EE641/Project
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
import torch
from torch.utils.data import Dataset, DataLoader
import torch.nn as nn
import torch.optim as optim
import torchvision
from torchvision import transforms
import matplotlib.pyplot as plt
import seg_models as sm
import torch.optim.lr_scheduler as lr_scheduler
from utils import *
device = (torch.device('cuda') if torch.cuda.is_available()
                  else torch.device('cpu'))
print(f"Training on device {device}.")

Training on device cuda.


In [3]:
path = "HDF5Data/lung_data.hdf5"
train_transforms = transforms.Compose([
                                transforms.RandomHorizontalFlip(p=0.5),
                                transforms.RandomRotation((-10, 10))
                                ])

train_img_transforms = transforms.Compose([transforms.Normalize((0.5127),(0.2474))])
test_img_transforms = transforms.Compose([transforms.Normalize((0.5127),(0.2474))])

train_dataset = HDF5Dataset(path, 'train', transform=train_transforms, img_transform=train_img_transforms)
val_dataset = HDF5Dataset(path, 'val', img_transform=test_img_transforms)

batch_size = 64
num_works=2
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=num_works)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=True, num_workers=num_works)

In [4]:
def train_lung_model(encoder_name, train_loader=train_loader, val_loader=val_loader,
                       n_epochs=15, stop=5):

    model = sm.Unet(
        encoder_name=encoder_name,
        encoder_weights='imagenet',
        in_channels=1,                  
        classes=1,
        decoder_channels = (256, 128, 64, 32, 16),
        activation=nn.Sigmoid                      
    ).to(device)

    # optimizer = optim.SGD(Unet_micronet_m0.parameters(), lr=0.001, momentum=0.9)
    optimizer = optim.RAdam(model.parameters(), lr=0.001)

    # scheduler1 = lr_scheduler.CosineAnnealingWarmRestarts(optimizer, T_0=10, T_mult=2)
    # scheduler2 = lr_scheduler.MultiStepLR(optimizer, milestones=[50, 100], gamma=0.1)
    # scheduler3 = lr_scheduler.StepLR(optimizer, step_size=25, gamma=0.1)
    # scheduler4 = None
    loss_fcn = DiceBCELoss()
    training_loop(
        model, optimizer,
        loss_fcn, train_loader,
        val_loader, scheduler = None,
        mask_name='lung mask',
        n_epochs=n_epochs, device=device, 
        save_path=f'trained_models/lung_Unet_{encoder_name}.pt',
        stop = stop)
    
    torch.cuda.empty_cache()

In [5]:
name_list = [f'micronet_m{i}' for i in range(4)]
name_list.extend(['resnet18', 'mobilenet_v2'])
for encoder_name in name_list:
    print(f"{encoder_name}:")
    train_lung_model(encoder_name)
    torch.cuda.empty_cache()

micronet_m0:


Downloading: "http://www.svcl.ucsd.edu/projects/micronet/assets/micronet-m0.pth" to /root/.cache/torch/hub/checkpoints/micronet-m0.pth


  0%|          | 0.00/3.97M [00:00<?, ?B/s]

Epoch 0, train loss: 0.476
Epoch 0, val loss: 0.139
Detect Improvement, Save Model
Epoch 1, train loss: 0.107
Epoch 1, val loss: 0.079
Detect Improvement, Save Model
Epoch 2, train loss: 0.082
Epoch 2, val loss: 0.071
Detect Improvement, Save Model
Epoch 3, train loss: 0.074
Epoch 3, val loss: 0.065
Detect Improvement, Save Model
Epoch 4, train loss: 0.070
Epoch 4, val loss: 0.062
Detect Improvement, Save Model
Epoch 5, train loss: 0.068
Epoch 5, val loss: 0.061
Detect Improvement, Save Model
Epoch 6, train loss: 0.066
Epoch 6, val loss: 0.061
Detect Improvement, Save Model
Epoch 7, train loss: 0.064
Epoch 7, val loss: 0.063
Epoch 8, train loss: 0.063
Epoch 8, val loss: 0.059
Detect Improvement, Save Model
Epoch 9, train loss: 0.063
Epoch 9, val loss: 0.058
Detect Improvement, Save Model
Epoch 10, train loss: 0.062
Epoch 10, val loss: 0.058
Epoch 11, train loss: 0.061
Epoch 11, val loss: 0.058
Epoch 12, train loss: 0.061
Epoch 12, val loss: 0.057
Detect Improvement, Save Model
Epoch 13

Downloading: "http://www.svcl.ucsd.edu/projects/micronet/assets/micronet-m1.pth" to /root/.cache/torch/hub/checkpoints/micronet-m1.pth


...

  0%|          | 0.00/44.7M [00:00<?, ?B/s]

Epoch 0, train loss: 0.317
Epoch 0, val loss: 0.093
Detect Improvement, Save Model
Epoch 1, train loss: 0.076
Epoch 1, val loss: 0.063
Detect Improvement, Save Model
Epoch 2, train loss: 0.065
Epoch 2, val loss: 0.059
Detect Improvement, Save Model
Epoch 3, train loss: 0.061
Epoch 3, val loss: 0.053
Detect Improvement, Save Model
Epoch 4, train loss: 0.059
Epoch 4, val loss: 0.052
Detect Improvement, Save Model
Epoch 5, train loss: 0.056
Epoch 5, val loss: 0.052
Detect Improvement, Save Model
Epoch 6, train loss: 0.056
Epoch 6, val loss: 0.052
Detect Improvement, Save Model
Epoch 7, train loss: 0.055
Epoch 7, val loss: 0.052
Detect Improvement, Save Model
Epoch 8, train loss: 0.054
Epoch 8, val loss: 0.051
Detect Improvement, Save Model
Epoch 9, train loss: 0.054
Epoch 9, val loss: 0.062
Epoch 10, train loss: 0.056
Epoch 10, val loss: 0.050
Detect Improvement, Save Model
Epoch 11, train loss: 0.053
Epoch 11, val loss: 0.049
Detect Improvement, Save Model
Epoch 12, train loss: 0.053
Epo

Downloading: "https://download.pytorch.org/models/mobilenet_v2-b0353104.pth" to /root/.cache/torch/hub/checkpoints/mobilenet_v2-b0353104.pth


  0%|          | 0.00/13.6M [00:00<?, ?B/s]

Epoch 0, train loss: 0.311
Epoch 0, val loss: 0.084
Detect Improvement, Save Model
Epoch 1, train loss: 0.076
Epoch 1, val loss: 0.065
Detect Improvement, Save Model
Epoch 2, train loss: 0.066
Epoch 2, val loss: 0.056
Detect Improvement, Save Model
Epoch 3, train loss: 0.062
Epoch 3, val loss: 0.054
Detect Improvement, Save Model
Epoch 4, train loss: 0.058
Epoch 4, val loss: 0.054
Detect Improvement, Save Model
Epoch 5, train loss: 0.059
Epoch 5, val loss: 0.052
Detect Improvement, Save Model
Epoch 6, train loss: 0.057
Epoch 6, val loss: 0.054
Epoch 7, train loss: 0.056
Epoch 7, val loss: 0.057
Epoch 8, train loss: 0.055
Epoch 8, val loss: 0.051
Detect Improvement, Save Model
Epoch 9, train loss: 0.055
Epoch 9, val loss: 0.052
Epoch 10, train loss: 0.054
Epoch 10, val loss: 0.051
Detect Improvement, Save Model
Epoch 11, train loss: 0.054
Epoch 11, val loss: 0.050
Detect Improvement, Save Model
Epoch 12, train loss: 0.056
Epoch 12, val loss: 0.050
Detect Improvement, Save Model
Epoch 13